<h1><center>Build <a href="https://howsmynhs.co.uk/">HowsMyNHS</a> website</center></h1>

This is the main script used to automatically generate the html for <a href="https://howsmynhs.co.uk/">HowsMyNHS</a>.

In [1]:
import build_website as bw
waiting_data = 'NHSwaiting.npy'
bed_data = 'NHSbeds.npy'

<h2>Generate Plots</h2>

In [ ]:
#bw.plotWaitingData(waiting_data)
bw.plotBedData(bed_data)

Generating graphs for the number of beds ... 

<h2>Make homepage</h2>

In [ ]:
bw.MakeHomepage(waiting_data, bed_data)

<h2>Make all the other pages</h2>

In [ ]:
bw.build_trust_pages(waiting_data, bed_data)